In [ ]:
import torch, torchvision
import mmseg
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
config_file = '/home/mmsegmentation/configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file = '/home/mmsegmentation/checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# test a single image
img = '/home/mmsegmentation/demo/demo.png'
result = inference_segmentor(model, img)

# show the results
show_result_pyplot(model, img, result, get_palette('cityscapes'))

In [ ]:
# Let's take a look at the dataset
import mmcv
import matplotlib.pyplot as plt

img = mmcv.imread('data/house2/images/1A4O4TGL19.jpg')
plt.figure(figsize=(8, 6))
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

In [ ]:
import os.path as osp
import numpy as np
from PIL import Image
from tqdm import tqdm
from tqdm import trange
from tqdm import tqdm
import time
from concurrent import futures

# convert dataset annotation to semantic segmentation map
data_root = 'data/house2'
img_dir = 'images'
ann_dir = 'labels'
# define class and plaette for better visualization
classes = ('backgrade', 'house')
palette = [[0, 0, 0], [255, 255, 255]]

def save_png(file):
      seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
      seg_img = Image.fromarray(seg_map).convert('P')
      seg_img.putpalette(np.array(palette, dtype=np.uint8))
      seg_img.save(osp.join(data_root, ann_dir, file.replace('.txt','.png')))

In [ ]:
tasks, results = [], []
with futures.ThreadPoolExecutor(max_workers=20) as executor:
    for file in tqdm(mmcv.scandir(osp.join(data_root, ann_dir), suffix='.txt')):
        tasks.append(executor.submit(save_png, file))
    for task in tqdm(futures.as_completed(tasks), total=len(tasks)):
        pass

In [ ]:
import os.path as osp
import numpy as np
from PIL import Image
from tqdm import tqdm

# convert dataset annotation to semantic segmentation map
data_root = 'data/house2'
img_dir = 'images'
ann_dir = 'labels'
# define class and plaette for better visualization
classes = ('backgrade', 'house')
palette = [[0, 0, 0], [255, 255, 255]]

def save_png(file):
      seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
      seg_img = Image.fromarray(seg_map).convert('P')
      seg_img.putpalette(np.array(palette, dtype=np.uint8))
      seg_img.save(osp.join(data_root, ann_dir, file.replace('.txt','.png')))
    
for file in tqdm(mmcv.scandir(osp.join(data_root, ann_dir), suffix='.txt')):
      seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
      seg_img = Image.fromarray(seg_map).convert('P')
      seg_img.putpalette(np.array(palette, dtype=np.uint8))
      seg_img.save(osp.join(data_root, ann_dir, file.replace('.txt', 
                                                         '.png')))

In [ ]:
# Let's take a look at the segmentation map we got
import matplotlib.patches as mpatches
img = Image.open('data/house2/labels/1A4O4TGL19.png')
plt.figure(figsize=(8, 6))
im = plt.imshow(np.array(img.convert('RGB')))

# create a patch (proxy artist) for every color 
patches = [mpatches.Patch(color=np.array(palette[i])/255., 
                          label=classes[i]) for i in range(2)]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
           fontsize='large')

plt.show()

In [ ]:
# split train/val set randomly
split_dir = 'splits'
mmcv.mkdir_or_exist(osp.join(data_root, split_dir))
filename_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_dir), suffix='.png')]
with open(osp.join(data_root, split_dir, 'train.txt'), 'w') as f:
  # select first 4/5 as train set
    train_length = int(len(filename_list)*29/30)
    f.writelines(line + '\n' for line in filename_list[:train_length])
with open(osp.join(data_root, split_dir, 'val.txt'), 'w') as f:
  # select last 1/5 as train set
    f.writelines(line + '\n' for line in filename_list[train_length:])

In [ ]:
import os.path as osp
import os
# convert dataset annotation to semantic segmentation map
data_root = 'data/house2'
img_dir = 'test_a'
ann_dir = 'labels'
# define class and plaette for better visualization
classes = ('backgrade', 'house')
palette = [[0, 0, 0], [255, 255, 255]]
split_dir = 'splits'
mmcv.mkdir_or_exist(osp.join(data_root, split_dir))
print(osp.join(data_root, split_dir))
print(osp.join(data_root, ann_dir))

filename_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, img_dir), suffix='.jpg')]

with open(osp.join(data_root, split_dir, 'test.txt'), 'w') as f:
  # select last 1/5 as train set
    f.writelines(line + '\n' for line in filename_list)

In [ ]:
len(filename_list)

In [ ]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class StandfordBackgroundDataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

    

In [ ]:
from mmcv import Config
cfg = Config.fromfile('configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes.py')

In [ ]:
from mmseg.apis import set_random_seed

# Since we use ony one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

# Modify dataset type and path
cfg.dataset_type = 'StandfordBackgroundDataset'
cfg.data_root = data_root

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu=8

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (256, 256)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(320, 240), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(320, 240),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_dir
cfg.data.train.ann_dir = ann_dir
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_dir
cfg.data.val.ann_dir = ann_dir
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_dir
cfg.data.test.ann_dir = ann_dir
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = '/home/mmsegmentation/checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.runner.max_iters = 200
cfg.log_config.interval = 10
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 200

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Build the detector
model = build_segmentor(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))


In [ ]:
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

In [ ]:
img = mmcv.imread('data/house2/images/1A4O4TGL19.jpg')

model.cfg = cfg
result = inference_segmentor(model, img)
plt.figure(figsize=(8, 6))
show_result_pyplot(model, img, result, palette)

In [ ]:
img1 = Image.open('data/house2/labels/1A7173V2GB.png')
img2 = Image.open('data/house2/images/1A7173V2GB.jpg')
img3 = Image.open('data/iccv09Data/labels/0000047.png')
img4 = Image.open('data/iccv09Data/images/0000047.jpg')


In [ ]:
print(img1.size)
print(img2.size)
print(img3.size)
print(img4.size)

In [ ]:
img1 = mmcv.imread('data/house2/labels/1A7173V2GB.png')
img2 = mmcv.imread('data/house2/images/1A7173V2GB.jpg')
img3 = mmcv.imread('data/iccv09Data/labels/0000047.png')
img4 = mmcv.imread('data/iccv09Data/images/0000047.jpg')

print(img1.shape)
print(img2.shape)
print(img3.shape)
print(img4.shape)

In [ ]:
import matplotlib.image as mpimg # mpimg 用于读取图片

img1 = mpimg.imread('data/house2/labels/1AXLWAONTR.png')
img2 = mpimg.imread('data/house2/images/1AXLWAONTR.jpg')
img3 = mpimg.imread('data/iccv09Data/labels/0000047.png')
img4 = mpimg.imread('data/iccv09Data/images/0000047.jpg')

print(img1.shape)
print(img2.shape)
print(img3.shape)
print(img4.shape)

In [ ]:
img1.shape

In [ ]:
img1[:,:,3]

In [ ]:
img1.shape[:2]

In [ ]:
      seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
      seg_img = Image.fromarray(seg_map).convert('P')
      seg_img.putpalette(np.array(palette, dtype=np.uint8))
      seg_img.save(osp.join(data_root, ann_dir, file.replace('.txt', 
                                                         '.png')))

In [ ]:
import numpy as np
import pandas as pd
import cv2

def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 将rle格式进行解码为图片
def rle_decode(mask_rle, shape=(512, 512)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')
#train_mask = pd.read_csv('/home/mmsegmentation/data/VOCdevkit/train_mask.csv', sep='\t', names=['name', 'mask'])



In [ ]:
a = train_mask['mask'].iloc[4745]

In [ ]:
mask = np.zeros(512*512, dtype=np.uint8)
mask.reshape((512, 512), order='F')

In [ ]:
train_mask['name'].sort_values()

In [ ]:
import os
len(os.listdir('data/house2/labels'))

In [ ]:
len(os.listdir('data/house2/images'))

In [ ]:
len(os.listdir('data/VOCdevkit/label/'))

In [ ]:
len(list(set(os.listdir('data/VOCdevkit/label/'))))

In [ ]:
import numpy as np
import pandas as pd
import cv2
train_mask = pd.read_csv('/home/mmsegmentation/data/VOCdevkit/train_mask.csv', sep='\t', names=['name', 'mask'])

In [ ]:
len(train_mask['mask'])

In [ ]:
from collections import Counter   #引入Counter

b = dict(Counter((os.listdir('data/VOCdevkit/label/'))))
[key for key,value in b.items()if value > 1]

In [ ]:
a = os.listdir('data/VOCdevkit/label/')
a.sort()

In [ ]:
from tqdm import trange
from tqdm import tqdm
import time
from concurrent import futures

def out_txt(i):
    try:
        mask = rle_decode(train_mask['mask'].iloc[i])
    except:
        mask = np.zeros(512*512, dtype=np.uint8)
        mask = mask.reshape((512, 512), order='F')

        path_txt = 'data/VOCdevkit/label2/' +  train_mask['name'].iloc[i].split('.')[0] + '.txt'
        np.savetxt(path_txt,mask, fmt="%i",delimiter=" ")

In [ ]:
tasks, results = [], []
with futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i in range(len(train_mask)):
        tasks.append(executor.submit(out_txt, i))
    for task in tqdm(futures.as_completed(tasks), total=len(tasks)):
        results.append(task.result())

In [ ]:
import os.path as osp
import numpy as np
from PIL import Image
from tqdm import tqdm

# convert dataset annotation to semantic segmentation map
data_root = 'data/house2'
img_dir = 'images'
ann_dir = 'labels'
# define class and plaette for better visualization
classes = ('backgrade', 'house')
palette = [[0, 0, 0], [255, 255, 255]]

def save_png(file):
      seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
      seg_img = Image.fromarray(seg_map).convert('P')
      seg_img.putpalette(np.array(palette, dtype=np.uint8))
      seg_img.save(osp.join(data_root, ann_dir, file.replace('.txt','.png')))

In [ ]:
tasks, results = [], []
with futures.ThreadPoolExecutor(max_workers=5) as executor:
    for file in tqdm(mmcv.scandir(osp.join(data_root, ann_dir), suffix='.txt')):
        tasks.append(executor.submit(save_png, file))
    for task in tqdm(futures.as_completed(tasks), total=len(tasks)):
        pass

In [ ]:
import pickle
f = open('house2_2/result.pkl','rb')
data = pickle.load(f)

In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
data[0]

In [ ]:
mask = []
name = []
for i,image in enumerate(data):
    rle = rle_encode(image.astype('uint8'))
    mask += [rle]
    if i%100 == 0:
        print(i)
    
df = pd.DataFrame()
test_mask = pd.read_csv('data/VOCdevkit/test_a_samplesubmit.csv',sep='\t',names = ['name','mask'])

name = test_mask['name'] 

df['name'] = name
mask_all = ['' for i in range(len(df))]

In [ ]:
filename_list[i] + '.jpg'

In [ ]:
for i in range(len(mask)):
    ind  = df[df.name==filename_list[i]+ '.jpg'].index[0]
    mask_all[ind] = mask[i]
df['mask'] = mask_all

df.to_csv('data/house2/test_b.csv',encoding='utf-8',header = None,index=False,sep = '\t')

In [ ]:
# Let's take a look at the segmentation map we got
import matplotlib.patches as mpatches
img = Image.open('data/house2/labels/1A4O4TGL19.png')

plt.figure(figsize=(8, 6))
im = plt.imshow(np.array(img.convert('RGB')))

# create a patch (proxy artist) for every color 
#patches = [mpatches.Patch(color=np.array(palette[i])/255., 
#                          label=classes[i]) for i in range(2)]
# put those patched as legend-handles into the legend
#plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
#           fontsize='large')

#plt.show()

In [ ]:
seg_img = Image.fromarray(data[0].astype('uint8')).convert('P')
seg_img.putpalette(np.array(palette, dtype=np.uint8))
seg_img

In [ ]:
data[0].astype('uint8')

In [ ]:
seg_map = np.loadtxt('data/house/labels/label/0A3B10OZ9S.txt').astype(np.uint8)

In [ ]:
seg_map

In [ ]:
import os
len(os.listdir('data/house2/images'))